# 04 Temp understand logic

In [ ]:
# set up connection to your google drive
# please click on the link generated and enter the authorisation code

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/Colab/statbot

In [ ]:
! pip install rank-bm25

In [ ]:
 # update spacy 
! pip install -U spacy
! python -m spacy info

In [ ]:
# load German model
! python -m spacy download de_core_news_lg

## Script A04
This script is very raw and was meant to test if the NERs could be used to identify which dataset the question is about. When someone asks a question, the bot is supposed to give back data from a dataset - but from which one? This is why a BM25-algorithm is used here, based on the dataset named-entities identified in the question. The script does this by checking it against the combination of the dataset titles in conjunction with relevant metadata that has been saved from the datasets.

## Thoughs for improvement
- Add an accuracy test with gold data
- Make any improvements you can think of

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("data/datasets_overview.csv")

In [ ]:
import numpy as np
import spacy
from rank_bm25 import BM25Okapi
from tqdm import tqdm
nlp = spacy.load("de_core_news_lg")
text_list = df["dataset_title"].str.lower().values
text_list=np.unique(text_list)
tok_text=[] # for our tokenised corpus
#Tokenising using SpaCy:
for doc in tqdm(nlp.pipe(text_list, disable=["tagger", "parser","ner"])):
    tok = [t.text for t in doc if t.is_alpha]
    tok_text.append(tok)

333it [00:00, 1554.52it/s]


In [ ]:
text_list

array(['1-personenhaushalte [anz.]', '15-jahresverbrauch  [%]',
       '15-jahresverbrauch [ha]', '2-personenhaushalte [anz.]',
       '3-personenhaushalte [anz.]', '4-personenhaushalte [anz.]',
       '5-personenhaushalte [anz.]',
       '6 und mehr personenhaushalte [anz.]',
       'abstimmungsarchiv des kantons zürich', 'aerzte [anz.]',
       'aerzte [pro 1000 einw.]', 'altersquotient [%]',
       'altpapiermengen [tonnen]',
       'anfragestatistik des data shop im statistischen amt des kantons zürich',
       'anteil 1 zi.-wohnungen [%]', 'anteil 1-personenhaushalte [%]',
       'anteil 2 zi.-wohnungen [%]', 'anteil 2-personenhaushalte [%]',
       'anteil 3 zi.-wohnungen [%]', 'anteil 3-personenhaushalte [%]',
       'anteil 4 zi.-wohnungen [%]', 'anteil 4-personenhaushalte [%]',
       'anteil 5 zi.-wohnungen [%]', 'anteil 5-personenhaushalte [%]',
       'anteil 6 und mehr personenhaushalte [%]',
       'anteil 6+ zi.-wohnungen [%]', 'anteil efh am wohnungsbestand [%]',
      

In [ ]:
bm25 = BM25Okapi(tok_text)

In [ ]:
#The following snipped gives back the five items that BM25 identified as being the most likely datasets

In [ ]:
query = "Eigenkapital"
tokenized_query = query.lower().split(" ")
import time
t0 = time.time()
results = bm25.get_top_n(tokenized_query, text_list, n=5)
t1 = time.time()
print(f'Searched in {round(t1-t0,3) } seconds \n')

for i in results:
    print(i)

Searched in 0.001 seconds 

eigenkapital [fr./einw.]
eigenkapital steuerpfl. jur. pers. [mio.fr.]
ø eigenkapital steuerpfl. jur. pers. [1000 fr.]
durchschnittliche haushaltsgrösse [pers.]
bevölkerungsdichte [einw./km2]
